# Extracción

Cargue de libreria

In [17]:
pip install sqlalchemy psycopg2-binary pandas

Note: you may need to restart the kernel to use updated packages.


Crear la conexión a PostgreSQL

In [18]:
from sqlalchemy import create_engine
import pandas as pd

# Parámetros de conexión
usuario = "fabio"         # ← Reemplaza por tu usuario de PostgreSQL
clave = "fabio"        # ← Reemplaza por tu contraseña
host = "localhost"             # ← Si está en tu máquina
puerto = "5432"                # Puerto por defecto de PostgreSQL
bd = "ProyectoMensajeria"         # Nombre de tu base de datos

# Crear el motor de conexión
engine = create_engine(f"postgresql+psycopg2://{usuario}:{clave}@{host}:{puerto}/{bd}")

Ver qué tablas tiene la base

Se realiza este paso para validar que tablas se cargaron de postgres

In [19]:
# Ver tablas disponibles
from sqlalchemy import inspect

inspector = inspect(engine)
tablas = inspector.get_table_names()
print("Tablas disponibles en la base:", tablas)

Tablas disponibles en la base: ['admin_interface_theme', 'authtoken_token', 'areas_cliente', 'auth_user_groups', 'auth_group', 'auth_user', 'auth_user_user_permissions', 'ciudad', 'cliente', 'clientes_coordinador', 'clientes_mensajeroaquitoy', 'clientes_mensajeroaquitoy_clientes', 'clientes_datosmensajero', 'area', 'auth_permission', 'django_admin_log', 'clientes_usuarioaquitoy', 'departamento', 'django_content_type', 'django_migrations', 'django_session', 'mensajeria_tiponovedad', 'mensajeria_servicio', 'mensajeria_estado', 'mensajeria_novedadesservicio', 'mensajeria_origenservicio', 'mensajeria_tipopago', 'mensajeria_estadosservicio', 'mensajeria_destinoservicio', 'mensajeria_tipovehiculo', 'mensajeria_tiposervicio', 'sede', 'auth_group_permissions', 'tipo_cliente', 'mensajeria_documentoasociado']


Extraer las tablas desde PostgreSQL a Pandas

In [20]:
# Tablas de dimensiones
df_cliente = pd.read_sql("SELECT * FROM cliente", engine)
df_sede = pd.read_sql("SELECT * FROM sede", engine)
df_ciudad = pd.read_sql("SELECT * FROM ciudad", engine)
df_tipo_cliente = pd.read_sql("SELECT * FROM tipo_cliente", engine)
df_mensajero = pd.read_sql("SELECT * FROM clientes_mensajeroaquitoy", engine) 
df_tipo_vehiculo = pd.read_sql("SELECT * FROM mensajeria_tipovehiculo", engine)
df_estado = pd.read_sql("SELECT * FROM mensajeria_estado", engine)
df_tipo_novedad = pd.read_sql("SELECT * FROM mensajeria_tiponovedad", engine)
df_tipo_servicio = pd.read_sql("SELECT * FROM mensajeria_tiposervicio", engine)
df_tipo_pago = pd.read_sql("SELECT * FROM mensajeria_tipopago", engine)
df_auth_user = pd.read_sql("SELECT * FROM auth_user", engine)

# Tablas de hechos / procesos
df_servicio = pd.read_sql("SELECT * FROM mensajeria_servicio", engine)
df_estados_servicio = pd.read_sql("SELECT * FROM mensajeria_estadosservicio", engine)
df_novedades = pd.read_sql("SELECT * FROM mensajeria_novedadesservicio", engine)
df_origen = pd.read_sql("SELECT * FROM mensajeria_origenservicio", engine)
df_destino = pd.read_sql("SELECT * FROM mensajeria_destinoservicio", engine)

Transformacion de tablas

In [21]:
import pandas as pd
from datetime import datetime, time

def calculate_time_diff(start_date_col, start_time_col, end_date_col, end_time_col):
    mask = (
        pd.notnull(start_date_col) & pd.notnull(start_time_col) & 
        pd.notnull(end_date_col) & pd.notnull(end_time_col)
    )
    start_dt = pd.to_datetime(
        start_date_col[mask].astype(str) + ' ' + start_time_col[mask].astype(str), 
        format='mixed', errors='coerce'
    )
    end_dt = pd.to_datetime(
        end_date_col[mask].astype(str) + ' ' + end_time_col[mask].astype(str), 
        format='mixed', errors='coerce'
    )
    diff = (end_dt - start_dt).dt.total_seconds() / 60
    result = pd.Series(index=start_date_col.index, dtype='float64')
    result[mask] = diff
    return result

In [22]:
# --- Dimensión: dim_hora ---
hours = [time(h, 0, 0) for h in range(24)]
dim_hora = pd.DataFrame(hours, columns=["Hora"])
dim_hora["ID_Hora"] = dim_hora.index + 1
dim_hora["Hour"] = dim_hora["Hora"].apply(lambda t: t.hour)

def get_periodo(hour):
    if 6 <= hour < 12:
        return "Mañana"
    elif 12 <= hour < 18:
        return "Tarde"
    elif 18 <= hour < 24:
        return "Noche"
    else:
        return "Madrugada"

def es_hora_laboral(hour):
    return 8 <= hour < 18

dim_hora["Periodo"] = dim_hora["Hour"].apply(get_periodo)
dim_hora["HoraLaboral"] = dim_hora["Hour"].apply(es_hora_laboral)
dim_hora = dim_hora[["ID_Hora", "Hora", "Hour", "Periodo", "HoraLaboral"]]
print("Dimensión dim_hora creada con", len(dim_hora), "filas.")

Dimensión dim_hora creada con 24 filas.


In [23]:
# --- Dimensión: dim_fecha ---
start_date = pd.to_datetime("2023-01-01")
end_date = pd.to_datetime("2025-12-31")
dates = pd.date_range(start=start_date, end=end_date, freq="D")
dim_fecha = pd.DataFrame(dates, columns=["Fecha"])
dim_fecha["ID_Fecha"] = dim_fecha.index + 1
dim_fecha["Year"] = dim_fecha["Fecha"].dt.year
dim_fecha["Month"] = dim_fecha["Fecha"].dt.month
dim_fecha["Day"] = dim_fecha["Fecha"].dt.day
dim_fecha["MonthName"] = dim_fecha["Fecha"].dt.month_name(locale="es_ES")
dim_fecha["DayName"] = dim_fecha["Fecha"].dt.day_name(locale="es_ES")
dim_fecha["Quarter"] = dim_fecha["Fecha"].dt.quarter
dim_fecha["IsWeekend"] = dim_fecha["Fecha"].dt.dayofweek.isin([5, 6])
dim_fecha = dim_fecha[["ID_Fecha", "Fecha", "Year", "Month", "Day", "MonthName", "DayName", "Quarter", "IsWeekend"]]
print("Dimensión dim_fecha creada con", len(dim_fecha), "filas.")


Dimensión dim_fecha creada con 1096 filas.


In [24]:
# --- Dimensión: dim_sede ---
# Unimos origen con sede para obtener información de la sede asociada al origen
dim_sede = df_origen.merge(df_sede, how="left", left_on="id", right_on="sede_id", suffixes=("_origen", "_sede"))
print("Columnas después del merge con sede:", dim_sede.columns.tolist(), "Filas:", len(dim_sede))
        
# Unimos con ciudad para obtener el nombre de la ciudad de la sede
dim_sede = dim_sede.merge(df_ciudad, how="left", left_on="ciudad_id_sede", right_on="ciudad_id", suffixes=("", "_ciudad"))
print("Columnas después del merge con ciudad:", dim_sede.columns.tolist(), "Filas:", len(dim_sede))
        
# Seleccionamos columnas relevantes y renombramos para claridad
expected_columns = ["id", "sede_id", "nombre", "ciudad_id_sede", "nombre_ciudad"]
available_columns = [col for col in expected_columns if col in dim_sede.columns]
if not all(col in dim_sede.columns for col in expected_columns):
    missing_columns = [col for col in expected_columns if col not in dim_sede.columns]
    print(f"Columnas faltantes en dim_sede: {missing_columns}")
dim_sede = dim_sede[available_columns].drop_duplicates(subset=["id"])
print("Dimensión dim_sede creada con", len(dim_sede), "filas.")

Columnas después del merge con sede: ['id', 'direccion_origen', 'cliente_id_origen', 'ciudad_id_origen', 'sede_id', 'nombre', 'direccion_sede', 'telefono', 'nombre_contacto', 'ciudad_id_sede', 'cliente_id_sede'] Filas: 741
Columnas después del merge con ciudad: ['id', 'direccion_origen', 'cliente_id_origen', 'ciudad_id_origen', 'sede_id', 'nombre', 'direccion_sede', 'telefono', 'nombre_contacto', 'ciudad_id_sede', 'cliente_id_sede', 'ciudad_id', 'nombre_ciudad', 'departamento_id'] Filas: 741
Dimensión dim_sede creada con 741 filas.


In [25]:
# --- Dimensión: dim_cliente ---
dim_cliente = df_cliente[["cliente_id", "nit_cliente", "nombre", "ciudad_id", "tipo_cliente_id"]]
dim_cliente = dim_cliente.rename(columns={
    "cliente_id": "ID_Cliente", "nit_cliente": "NIT_Cliente", "nombre": "Nombre_Cliente",
    "ciudad_id": "ID_Ciudad", "tipo_cliente_id": "ID_Tipo_Cliente"
}).drop_duplicates(subset=["ID_Cliente"])
print("Dimensión dim_cliente creada con", len(dim_cliente), "filas.")


Dimensión dim_cliente creada con 27 filas.


In [26]:
# --- Dimensión: dim_mensajero ---

# Unir df_mensajero con df_auth_user para obtener el nombre de usuario
dim_mensajero = df_mensajero.merge(
	df_auth_user, how="left", left_on="user_id", right_on="id"
)
dim_mensajero = dim_mensajero[["id_x", "user_id", "username"]].rename(columns={
	"id_x": "ID_Mensajero", "user_id": "ID_User", "username": "Nombre_Mensajero"
}).drop_duplicates(subset=["ID_Mensajero"])
print("Dimensión dim_mensajero creada con", len(dim_mensajero), "filas.")

Dimensión dim_mensajero creada con 50 filas.


In [27]:
# --- Dimensión: dim_estado_servicio ---
dim_estado_servicio = df_estado[["id", "nombre"]].rename(columns={
    "id": "ID_Estado", "nombre": "Nombre_Estado"
})
dim_estado_servicio["Nombre_Estado"] = dim_estado_servicio["Nombre_Estado"].fillna("Desconocido")
dim_estado_servicio = dim_estado_servicio.drop_duplicates(subset=["ID_Estado"])
print("Dimensión dim_estado_servicio creada con", len(dim_estado_servicio), "filas.")

Dimensión dim_estado_servicio creada con 6 filas.


In [28]:
# --- Dimensión: dim_tipo_servicio ---
dim_tipo_servicio = df_tipo_servicio[["id", "nombre"]].rename(columns={
    "id": "ID_Tipo_Servicio", "nombre": "Nombre_Tipo_Servicio"
})
dim_tipo_servicio["Nombre_Tipo_Servicio"] = dim_tipo_servicio["Nombre_Tipo_Servicio"].fillna("Desconocido")
dim_tipo_servicio = dim_tipo_servicio.drop_duplicates(subset=["ID_Tipo_Servicio"])
print("Dimensión dim_tipo_servicio creada con", len(dim_tipo_servicio), "filas.")


Dimensión dim_tipo_servicio creada con 4 filas.


Tabla de hechos

In [29]:
# --- Tabla de hechos: T_hechos ---
T_hechos = df_servicio.rename(columns={"id": "ID_Servicio"})
print("Columnas iniciales de mensajeria_servicio:", T_hechos.columns.tolist(), "Filas:", len(T_hechos))

# Uniones con dimensiones relevantes
T_hechos = (
    T_hechos
    .merge(dim_cliente, left_on="cliente_id", right_on="ID_Cliente", how="left")
    .merge(dim_mensajero, left_on="mensajero_id", right_on="ID_Mensajero", how="left")
    .merge(dim_sede, left_on="origen_id", right_on="id", how="left", suffixes=("", "_origen"))
    .merge(dim_sede, left_on="destino_id", right_on="id", how="left", suffixes=("_origen", "_destino"))
    .merge(dim_tipo_servicio, left_on="tipo_servicio_id", right_on="ID_Tipo_Servicio", how="left")
)

# Unir con estados de servicio
T_hechos = T_hechos.merge(
    df_estados_servicio, left_on="ID_Servicio", right_on="servicio_id", how="left"
)

# Procesamiento de fechas y horas
T_hechos["fecha"] = pd.to_datetime(T_hechos["fecha"], errors='coerce')
T_hechos["hora"] = pd.to_datetime(T_hechos["hora"], format='%H:%M:%S', errors='coerce').dt.time
print("Valores nulos en fecha y hora:", T_hechos[["fecha", "hora"]].isnull().sum())

# Pivot para estados
T_hechos["id_estado"] = T_hechos["estado_id"].astype(str).replace("nan", None)
T_hechos["DateTimeValue"] = T_hechos.apply(
    lambda row: f"{row['fecha']} {row['hora']}" if pd.notnull(row['fecha']) and pd.notnull(row['hora']) else None, axis=1
)
pivot = T_hechos.pivot_table(
    index=["ID_Servicio", "ID_Cliente", "ID_Mensajero", "id_origen", "id_destino", "ID_Tipo_Servicio"],
    columns="id_estado",
    values="DateTimeValue",
    aggfunc="last"
).reset_index()
print("Columnas después de pivot_table:", pivot.columns.tolist(), "Filas:", len(pivot))

# Validación de estados esperados
expected_states = ["1", "2", "3", "4", "5", "6"]
missing = [e for e in expected_states if e not in pivot.columns]
if missing:
    print(f"Estados faltantes en la pivot table: {missing}")
estado_map = {str(k): v for k, v in {
    1: "Iniciado", 2: "Asignado", 3: "EnTransito", 4: "Recogido", 5: "Entregado", 6: "Terminado"
}.items()}
# Extraer fechas y horas por estado
for estado in expected_states:
    if estado in pivot.columns:
        nombre = estado_map[estado]
        pivot[f"Fecha_{nombre}"] = pd.to_datetime(pivot[estado], errors='coerce').dt.date
        pivot[f"Hora_{nombre}"] = pd.to_datetime(pivot[estado], errors='coerce').dt.time
pivot.drop(columns=[e for e in expected_states if e in pivot.columns], inplace=True)
print("Columnas después de añadir fechas y horas:", pivot.columns.tolist(), "Filas:", len(pivot))

# Cálculo de diferencias de tiempo
pivot["tiempo_total_min"] = calculate_time_diff(
    pivot["Fecha_Iniciado"], pivot["Hora_Iniciado"], 
    pivot["Fecha_Terminado"], pivot["Hora_Terminado"]
)
pivot["tiempo_asignacion_min"] = calculate_time_diff(
    pivot["Fecha_Iniciado"], pivot["Hora_Iniciado"], 
    pivot["Fecha_Asignado"], pivot["Hora_Asignado"]
)
pivot["tiempo_recoleccion_min"] = calculate_time_diff(
    pivot["Fecha_Asignado"], pivot["Hora_Asignado"], 
    pivot["Fecha_Recogido"], pivot["Hora_Recogido"]
)
pivot["tiempo_entrega_min"] = calculate_time_diff(
    pivot["Fecha_Recogido"], pivot["Hora_Recogido"], 
    pivot["Fecha_Entregado"], pivot["Hora_Entregado"]
)
pivot["tiempo_cierre_min"] = calculate_time_diff(
    pivot["Fecha_Entregado"], pivot["Hora_Entregado"], 
    pivot["Fecha_Terminado"], pivot["Hora_Terminado"]
)
print("Tabla T_hechos creada con", len(pivot), "filas.")

# Diccionario de datos transformados
data_trans = {
    "dim_hora": dim_hora,
    "dim_fecha": dim_fecha,
    "dim_sede": dim_sede,
    "dim_cliente": dim_cliente,
    "dim_mensajero": dim_mensajero,
    "dim_estado_servicio": dim_estado_servicio,
    "dim_tipo_servicio": dim_tipo_servicio,
    "T_hechos": pivot
}
print("Tabla de hechos generada:", {k: len(v) for k, v in data_trans.items()})


Columnas iniciales de mensajeria_servicio: ['ID_Servicio', 'descripcion', 'nombre_solicitante', 'fecha_solicitud', 'hora_solicitud', 'fecha_deseada', 'hora_deseada', 'nombre_recibe', 'telefono_recibe', 'descripcion_pago', 'ida_y_regreso', 'activo', 'novedades', 'cliente_id', 'destino_id', 'mensajero_id', 'origen_id', 'tipo_pago_id', 'tipo_servicio_id', 'tipo_vehiculo_id', 'usuario_id', 'prioridad', 'ciudad_destino_id', 'ciudad_origen_id', 'hora_visto_por_mensajero', 'visto_por_mensajero', 'descripcion_multiples_origenes', 'mensajero2_id', 'mensajero3_id', 'multiples_origenes', 'asignar_mensajero', 'es_prueba', 'descripcion_cancelado'] Filas: 28430
Valores nulos en fecha y hora: fecha     0
hora     51
dtype: int64
Columnas después de pivot_table: ['ID_Servicio', 'ID_Cliente', 'ID_Mensajero', 'id_origen', 'id_destino', 'ID_Tipo_Servicio', '1', '2', '3', '4', '5', '6'] Filas: 19984


C:\Users\fabio\AppData\Local\Temp\ipykernel_14024\1607933453.py:50: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pivot[f"Fecha_{nombre}"] = pd.to_datetime(pivot[estado], errors='coerce').dt.date
C:\Users\fabio\AppData\Local\Temp\ipykernel_14024\1607933453.py:51: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pivot[f"Hora_{nombre}"] = pd.to_datetime(pivot[estado], errors='coerce').dt.time
C:\Users\fabio\AppData\Local\Temp\ipykernel_14024\1607933453.py:50: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pivot[f"Fecha_{nombre}"] = pd.to_datetime(pivot[estado], errors='coerce').d

Columnas después de añadir fechas y horas: ['ID_Servicio', 'ID_Cliente', 'ID_Mensajero', 'id_origen', 'id_destino', 'ID_Tipo_Servicio', 'Fecha_Iniciado', 'Hora_Iniciado', 'Fecha_Asignado', 'Hora_Asignado', 'Fecha_EnTransito', 'Hora_EnTransito', 'Fecha_Recogido', 'Hora_Recogido', 'Fecha_Entregado', 'Hora_Entregado', 'Fecha_Terminado', 'Hora_Terminado'] Filas: 19984
Tabla T_hechos creada con 19984 filas.
Tabla de hechos generada: {'dim_hora': 24, 'dim_fecha': 1096, 'dim_sede': 741, 'dim_cliente': 27, 'dim_mensajero': 50, 'dim_estado_servicio': 6, 'dim_tipo_servicio': 4, 'T_hechos': 19984}


In [30]:
T_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128402 entries, 0 to 128401
Data columns (total 64 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   ID_Servicio                     128402 non-null  int64         
 1   descripcion                     128402 non-null  object        
 2   nombre_solicitante              128402 non-null  object        
 3   fecha_solicitud                 128402 non-null  object        
 4   hora_solicitud                  128402 non-null  object        
 5   fecha_deseada                   128402 non-null  object        
 6   hora_deseada                    128402 non-null  object        
 7   nombre_recibe                   128402 non-null  object        
 8   telefono_recibe                 128402 non-null  object        
 9   descripcion_pago                127012 non-null  object        
 10  ida_y_regreso                   128402 non-null  bool   

Carga de Dimensiones y tabla de hechos a POSTGRES

In [31]:
from sqlalchemy import create_engine

usuario = "fabio"
clave = "fabio"
host = "localhost"
puerto = "5432"
bd = "DimProyecto"

engine = create_engine(f"postgresql+psycopg2://{usuario}:{clave}@{host}:{puerto}/{bd}")


In [32]:

# Carga de tablas
for nombre_tabla, df in data_trans.items():
    df.to_sql(nombre_tabla, engine, if_exists='replace', index=False)
    print(f"Tabla '{nombre_tabla}' cargada")


Tabla 'dim_hora' cargada
Tabla 'dim_fecha' cargada
Tabla 'dim_sede' cargada
Tabla 'dim_cliente' cargada
Tabla 'dim_mensajero' cargada
Tabla 'dim_estado_servicio' cargada
Tabla 'dim_tipo_servicio' cargada
Tabla 'T_hechos' cargada
